# Figure 1. common variant results

Figure 1 combines the common variant results from:
- MOSTEST (A+B)
- Brainspan (from FUMA -> MAGMA) (C+D)
- Betas from mostest for the 3 lead SNPs (E+F)

In [1]:
import pandas as pd
import numpy as np
import os
import glob
import h5py
import scipy.io as sio
import scipy.stats as stats
import matplotlib.pyplot as plt
import re
import seaborn as sns
from nilearn import plotting, image
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from matplotlib import colors as mcolors, path

## Data loading

In [2]:
def add_row(df, new_data, column_names, index_name, np_array=True):
    if np_array:
        df_new = pd.DataFrame(data=new_data.reshape((1, len(column_names))), columns=column_names, index=[index_name])
    else:
        df_new = pd.DataFrame.from_dict(new_data)
    return pd.concat([df, df_new], axis=0)
        
def get_snp(chr_no, snp_id):
    """
    """
    bim_fn = "/data/clusterfs/lag/users/jitame/SENT_CORE/geno/mostest/in/mostest_geno_in_c{}.bim".format(chr_no)
    bim = pd.read_csv(bim_fn, sep='\t', header=None, names='CHR SNP GP BP A1 A2'.split())
    snp = bim.SNP.values == snp_id
    return snp, bim.loc[bim.SNP.values == snp_id, :].to_dict()

def get_file_name_mat(cat, chr_no):
    if cat == "edges":
        return "/data/clusterfs/lag/users/jitame/SENT_CORE/geno/mostest/out/sent_edges/edges_chr{}_zmat.mat".format(chr_no)
    if cat == "edges_HD":
        return "/data/clusterfs/lag/users/jitame/SENT_CORE/geno/mostest/out/sent_edges_asym/edges_asym_chr{}_zmat.mat".format(chr_no)

def get_betas_pvals(fname, snps):
    with h5py.File(fname, 'r') as h5file:
        beta_orig = np.array(h5file['beta_orig'])
        zmat_orig=np.array(h5file['zmat_orig'])
        pval_orig = stats.norm.sf(np.abs(zmat_orig))*2.0

        betas_snp = beta_orig[:, snps]
        pval_snp = pval_orig[:, snps]
    
        return betas_snp, pval_snp
    
def data_munger_MOSTEST_betas(snp_df, col_names, cat):
    """
    Load underlying beta's from MOSTEST analysis
    """
    
    beta_df = pd.DataFrame(columns=col_names)
    pval_df = pd.DataFrame(columns=col_names)
    snp_info_df = pd.DataFrame(columns='CHR SNP GP BP A1 A2'.split())
    
    for snp_no in snp_df.index.values:
        snp, snp_all = get_snp(snp_df.chr_no[snp_no], snp_df.ID[snp_no])
    
        betas_snp, pval_snp = get_betas_pvals(fname=get_file_name_mat(cat, snp_df.chr_no[snp_no]),
                                               snps=snp)
        
        beta_df = add_row(df = beta_df,
                          new_data = betas_snp,
                          column_names = col_names,
                          index_name = snp_df.ID[snp_no])
        
        pval_df = add_row(df = pval_df,
                          new_data = pval_snp,
                          column_names = col_names,
                          index_name = snp_df.ID[snp_no])
        
        snp_info_df = add_row(df = snp_info_df,
                              new_data = snp_all,
                              column_names = 'CHR SNP GP BP A1 A2'.split(),
                              index_name = snp_df.ID[snp_no],
                              np_array=False)
    
    return beta_df, pval_df, snp_info_df

In [3]:
def read_with_gzip(fn):
    return pd.read_csv(fn, sep="\t", compression="gzip", engine="pyarrow")

def get_cumulative_pos(df):
    running_pos = float(0)

    cumulative_pos = []

    for chrom, group_df in df.groupby('CHR'):
        chr_df = np.add(group_df['BP'].astype("int64"), running_pos)
        cumulative_pos.append(chr_df)
        running_pos += float(group_df['BP'].max())

    df['cumulative_pos'] = pd.concat(cumulative_pos)
    
    return df

def load_MOSTEST_sumstats(fn, out_fn=None):
    
    data = pd.read_csv(fn, sep="\t", compression="gzip", engine="pyarrow")
    
    data = data[data["CHR"]!="CHR"]
    
    data.loc[data["CHR"] == "X", "CHR" ] = 23
    data.loc[data["CHR"] == "XY", "CHR" ] = 23
    
    data["CHR"] = data["CHR"].astype("int32")
    data["BP"] = data["BP"].astype("int64")
    data["PVAL"] = data["PVAL"].astype("float64")
    data["Z"] = data["Z"].astype("float32")
    data["N"] = data["N"].astype("float32")
    
    data.sort_values(by=["CHR", "BP"], inplace=True)
    
    data.reset_index(drop=True, inplace=True)
    
    #get cumulative position
    data = get_cumulative_pos(data)
    
    data["LOGP"] = -np.log10(data["PVAL"].astype("float64"))

    if out_fn is not None:
        data.to_csv(out_fn, sep="\t", compression="gzip")
    
    return data

In [4]:
def MAGMA_data_loader_bs(fn):
    return pd.read_csv(fn, delim_whitespace=True, skiprows=5, header=0)

def process_bs(fn):
    data = MAGMA_data_loader_bs(fn)
    data["LOGP"] = -np.log10(data["P"])
    return data

## Plotting functions

In [5]:
def load_column_names_first_line(fn):
    return [str(x) for x in open( fn ).read().split('\n')[0].split("\t") ]

def load_column_names(fn):
    return [str(x) for x in open( fn ).read().split('\n')[:-1] ] 

def get_edge_names(list_names):
    """
    Returns edge combinations for edges
    """
    df_names=pd.DataFrame(index=list_names, columns=list_names)
    for i in range(len(list_names)):
        for j in range(len(list_names)):
            df_names.iloc[i, j] = list_names[i]+"*"+list_names[j]
            
    #heritable = load_column_names("/data/clusterfs/lag/users/jitame/SENT_CORE/heritable_edges.txt") + load_column_names("/data/clusterfs/lag/users/jitame/SENT_CORE/heritable_edges_asym.txt")
    #heritable = ["sent_edges_"+x for x in heritable]
    
    return [x for x in list(df_names.to_numpy()[np.triu_indices(len(list_names), k=1)].flatten()) if x in heritable]


def get_idx(cat):
    """
    Returns indices for category
    """
    sent_core_l_ind = [2, 30, 32, 40, 56, 98, 102, 146, 148, 166, 168, 170, 172, 174, 182, 184, 222, 224]
    sent_core_r_ind = [3, 31, 33, 41, 57, 99, 103, 147, 149, 167, 169, 171, 173, 175, 183, 185, 223, 225]
    sent_core_bi_ind = sorted(sent_core_l_ind + sent_core_r_ind)
    
    if cat == "edges":
        return sent_core_bi_ind
    elif cat == "edges_HD":
        return sent_core_l_ind
    
    
def set_up_coordinates(cat):
    """
    get coordinates for plot
    """
    #set up atlas + coordinates
    aicha_atlas = "/data/workspaces/lag/workspaces/lg-ukbiobank/projects/multilateral/FuncNet_AICHA/segs/AICHA.nii"
    aicha_img = image.load_img(aicha_atlas)
    aicha_coords = plotting.find_parcellation_cut_coords(aicha_img)
    
    #indices
    idx = get_idx(cat)

    #get coordinates and names
    return aicha_coords[idx, :]

def get_names_aicha(cat):
    """
    Return node names
    """
    aicha_test = "/data/clusterfs/lag/projects/lg-ukbiobank/working_data/imaging_data/AICHA/1000099/AICHA_timeseries_NO_GSR_cormat.txt"
    aicha2 = pd.read_csv(aicha_test, sep=";", index_col=0)
    
    idx = get_idx(cat)
    
    if cat == "edges":
        return list(aicha2.columns[idx]) #[x for x in list(aicha2.columns[idx]) if x in heritable]
    elif cat == "edges_HD":
        return list(x[:-2] for x in aicha2.columns[idx]) #[x for x in list(aicha2.columns[idx]) if x in heritable]


def make_nice_mat(input_data, cat, var_name):
    """
    put data from column into right spot in matrix
    """
    #get indices
    ind_names = get_names_aicha(cat)
        
    #set up dataframe
    df_out = pd.DataFrame(index=ind_names, columns=ind_names)
    df_out.iloc[np.diag_indices(len(ind_names)), np.diag_indices(len(ind_names))] = 0
    
    #put data in right place
    for x in list(input_data.columns):
        nodes = x.split("*", 1)
        df_out.loc[nodes[0], nodes[1]] = input_data.loc[var_name, x]
        df_out.loc[nodes[1], nodes[0]] = input_data.loc[var_name, x]
    
    return df_out
        
    
def plot_results_brain_betas(betas, pvals, cat, snp, title, vmax_in=None, alpha=None, ax=None, out=None, rare=False):
    """
    Main function:
    - loads data
    - significance testing
    - plot using nilearn
    """
    #get correlation values
    beta_mat = make_nice_mat(betas, cat, snp)
    
    if alpha:
        p_mat = make_nice_mat(pvals, cat, snp)
        beta_mat = beta_mat[p_mat < alpha]
        
    if vmax_in is None:
        vmax_in = np.max(np.max(beta_mat))
    
    #plot
    if np.abs(beta_mat.sum().sum()) < 0.0001:
        plotting.plot_markers([1]*len(beta_mat),
                              set_up_coordinates(cat), 
                              node_cmap="binary",
                              node_size=30,
                              alpha=0.8, 
                              display_mode='lyrz',
                              title=title, 
                              colorbar=False,
                              axes=ax)
        
    else:
        if rare:
            colors = ["whitesmoke", "indianred"]
            cmap1 = LinearSegmentedColormap.from_list("mycmap", colors)
            
            plotting.plot_connectome(beta_mat.to_numpy(dtype=float),
                             set_up_coordinates(cat),
                             title=title,
                             edge_cmap=cmap1,
                             edge_vmin=0,
                             edge_vmax=vmax_in,
                             node_color="dimgrey",
                             node_size=30,
                             display_mode="lyrz",
                             colorbar=True,
                             alpha=0.8,
                             axes=ax)
                             #output_file=out)    
        else:
            colors = ["mediumblue", "cornflowerblue", "lightgrey", "whitesmoke", "lightgrey", "lightcoral", "indianred"]
            cmap1 = LinearSegmentedColormap.from_list("mycmap", colors)
            plotting.plot_connectome(beta_mat.to_numpy(dtype=float),
                             set_up_coordinates(cat),
                             title=title,
                             edge_vmin=-vmax_in,
                             edge_vmax=vmax_in,
                             edge_cmap=cmap1,
                             node_color="dimgrey",
                             node_size=30,
                             display_mode="lyrz",
                             colorbar=True,
                             alpha=0.8,
                             axes=ax)
                             #output_file=out)    



In [6]:
def manhattan(df, ax, figargs={"figsize":(14,8),"dpi":300}):
    
    g = sns.scatterplot(
        data = df,
        x = 'cumulative_pos',
        y = 'LOGP',
        hue = 'CHR',
        palette = ['indianred', 'mediumblue'] * 11 + ['indianred'],
        linewidth=0,
        s=6,
        zorder=2,
        ax=ax,
        legend=None,
        style=None
    )
    
    alpha = -np.log10(0.05/10000000)
    
    ax.plot([np.min(df["cumulative_pos"]), np.max(df["cumulative_pos"])], [alpha, alpha], "--", color="navy")

    ax.set_xlabel('Chromosome')
    ax.set_ylabel('-Log10 P value')

    ax.set_xticks(df.groupby('CHR')['cumulative_pos'].median())
    ax.tick_params(which='major', width=0.3, length=2, labelsize=8)

    ax.set_xticklabels(df['CHR'].unique())
    
    ax.set_ylim(0, 50)
    
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.spines["left"].set_visible(True)
    ax.spines["bottom"].set_visible(True)

In [7]:
def gradientbars(bars, ydata, cmap):
    """
    https://towardsdatascience.com/beautiful-bars-scaled-gradient-fill-on-bar-plots-e4da4cdae033
    """
    ax = bars[0].axes
    lim = ax.get_xlim()+ax.get_ylim()
    ax.axis(lim)
    for bar in bars:
        bar.set_facecolor("none")
        x,y = bar.get_xy()
        w, h = bar.get_width(), bar.get_height()
        grad = np.atleast_2d(np.linspace(0,1*h/max(ydata),256)).T
        ax.imshow(grad, extent=[x,x+w,y,y+h], origin='lower', aspect="auto", 
                  norm=cm.colors.NoNorm(vmin=0,vmax=1), cmap=plt.get_cmap(cmap))

def brainspan_plot(brainspan_data, ax):
    colors = ["lightgray", "indianred"]
    cmap1 = LinearSegmentedColormap.from_list("mycmap", colors)
    mybar1 = ax.bar(brainspan_data.index.values, brainspan_data["LOGP"], width=0.6)
    ax.set_xticks(brainspan_data.index.values, brainspan_data.VARIABLE, rotation=90)
    ax.set_xlabel("Brainspan age")
    ax.set_ylabel("-Log10 P-value of association")
    ax.axhline(-np.log10(0.05/(len(brainspan_data))),color='navy',ls='--') 
    ax.set_ylim(0, 3.5)
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.spines["left"].set_visible(True)
    ax.spines["bottom"].set_visible(True)
    gradientbars(mybar1, brainspan_data["LOGP"], cmap1)

## Make figure 1

#### Load data

In [8]:
base_path = "/data/clusterfs/lag/users/jitame/SENT_CORE/"
results_path = "/data/workspaces/lag/workspaces/lg-ukbiobank/projects/rest-multimodal/results/"

#mostest mvgwas data
mvgwas_lang = load_MOSTEST_sumstats(os.path.join(results_path, "sumstats", "edges.sumstats.gz"))
mvgwas_asym = load_MOSTEST_sumstats(os.path.join(results_path, "sumstats", "edges_asym.sumstats.gz"))

#load data
brainspan_lang = process_bs(os.path.join(results_path, "FUMA", "edges_FUMA", "magma_exp_bs_age_avg_log2RPKM.gsa.out"))
brainspan_asym = process_bs(os.path.join(results_path, "FUMA", "edges_asym_FUMA", "magma_exp_bs_age_avg_log2RPKM.gsa.out"))

#top variants
#lang
common_top_lang = pd.DataFrame.from_dict({"chr_no":[3,3,10], "ID":["rs35124509", "rs2279829", "rs2274224"]})
col_names_lang = load_column_names_first_line("/data/clusterfs/lag/users/jitame/SENT_CORE/pheno/sent_edges_N29681_resid_mostest.txt")
beta_lang, p_lang, snp_lang = data_munger_MOSTEST_betas(common_top_lang, col_names=col_names_lang, cat="edges")

#asym
common_top_asym = pd.DataFrame.from_dict({"chr_no":[3,3,3], "ID":["rs7635916", "rs2279829", "rs13321297"]})
col_names_asym = load_column_names_first_line("/data/clusterfs/lag/users/jitame/SENT_CORE/pheno/sent_edges_asym_N29681_resid_mostest.txt")
beta_asym, p_asym, snp_asym = data_munger_MOSTEST_betas(common_top_asym, col_names=col_names_asym, cat="edges_HD")

#### Plotting

In [ ]:
#plot_path = "/data/workspaces/lag/workspaces/lg-ukbiobank/projects/rest-multimodal/results"

fig_args = {"figsize":(17.8,22),"dpi":400}

fig, ax = plt.subplots(5, 2, gridspec_kw={'height_ratios': [1.4, 1.2, 1, 1, 1]}, **fig_args)

vmax_in = float(0.088)
vmax_in_hd = float(0.043)

#A
manhattan(mvgwas_lang, ax=ax[0,0])

#B
brainspan_plot(brainspan_lang, ax=ax[1,0])

#C
for x, snp in enumerate(beta_lang.index.values):
    plot_results_brain_betas(betas=beta_lang,
                             pvals=p_lang,
                             vmax_in = vmax_in,
                             cat="edges",
                             ax=ax[x+2,0],
                             title="{0} (chr{1}_{2}_{3}:{4})".format(snp,
                                                                   snp_lang.loc[snp_lang.SNP.values == snp, "CHR"].to_string(index=False),
                                                                   snp_lang.loc[snp_lang.SNP.values == snp, "BP"].to_string(index=False),
                                                                   snp_lang.loc[snp_lang.SNP.values == snp, "A1"].to_string(index=False),
                                                                   snp_lang.loc[snp_lang.SNP.values == snp, "A2"].to_string(index=False)),
                             snp=snp)

#D
manhattan(mvgwas_asym, ax=ax[0,1])

#E
brainspan_plot(brainspan_asym, ax=ax[1,1])

#F
for x, snp in enumerate(beta_asym.index.values):
    plot_results_brain_betas(betas=beta_asym,
                             pvals=p_asym,
                             cat="edges_HD",
                             vmax_in = vmax_in_hd,
                             ax=ax[x+2,1],
                             snp=snp,
                             title="{0} (chr{1}_{2}_{3}:{4})".format(snp,
                                                                     snp_asym.loc[snp_asym.SNP.values == snp, "CHR"].to_string(index=False),
                                                                     snp_asym.loc[snp_asym.SNP.values == snp, "BP"].to_string(index=False),
                                                                     snp_asym.loc[snp_asym.SNP.values == snp, "A1"].to_string(index=False),
                                                                     snp_asym.loc[snp_asym.SNP.values == snp, "A2"].to_string(index=False)))


plt.subplots_adjust(hspace=0.5, wspace=0.1)

gwas_ticks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,"",17,"",19,"",21,"","X"]

#Titles
#fig.suptitle("Common variant results", y=0.95, fontsize=28)
ax[0, 0].text(50, 61, "Language network", fontsize=20)
ax[0, 0].set_xticks(ax[0,0].get_xticks(), gwas_ticks)
ax[0, 0].text(0, 55, "A. mvGWAS", fontsize=16)
ax[1, 0].text(0, 3.85, "C. Brainspan MAGMA", fontsize=16)
ax[2, 0].text(0.06, 1.08, "E. Betas lead variants", fontsize=16)
ax[0, 1].text(50, 61, "Hemispheric differences", fontsize=20)
ax[0, 1].set_xticks(ax[0,1].get_xticks(), gwas_ticks)
ax[0, 1].text(0, 55, "B. mvGWAS", fontsize=16)
ax[1, 1].text(0, 3.85, "D. Brainspan MAGMA", fontsize=16)
ax[2, 1].text(0.06, 1.08, "F. Betas lead variants", fontsize=16)


#plt.savefig(fname=os.path.join(results_path, "Figure_1_common_results_v3.tif"), bbox_inches="tight")
#plt.savefig(fname=os.path.join(results_path, "Figure_1_common_results_v3.eps"), bbox_inches="tight")
plt.savefig(fname=os.path.join(results_path, "Figure_1_common_results_v3.pdf"), bbox_inches="tight")
plt.savefig(fname=os.path.join(results_path, "Figure_1_common_results_v3.png"), bbox_inches="tight")